# StateLegiscraper: PDF Format Example Notebook

*Author*: Katherine Chang (kachang@uw.edu)

*Last Updated*: 12/8/2021

StateLegiscraper is a Python package that scrapes and processes data from U.S. state legislature websites. As of writing, the package is focused on transcribing standing committee hearings from each state legislature from its native archival format to text, so that this text data can be easily used for NLP research purposes and for public review. For more details about the StateLegiscraper, please visit its [Github repository](https://github.com/ka-chang/StateLegiscraper) where it is under active development. 

This notebook walks a new user through the StateLegiscraper workflow, with a focus on the Nevada State Legislature and working with PDF file formats. 

This notebook makes several assumptions about the user, which are that they have:

- At least a novice level familiarity with Python, including importing packages, running basic functions, and saving files.
- Knowledge of different Python file types, particularily lists and dictionaries. 
- Comfort working in the command line, as StateLegiscraper is installed through the user's choice of terminal. 
- Have at least 100 mb of space on their local hard drive or a mounted cloud drive to save the raw data on.

## The Nevada Context

The Nevada State Legislature is a part-time biennial state legislature, which means state legislators meet on odd number of years between the months of February to June. The state legislature website, [www.leg.state.nv.us](www.leg.state.nv.us), hosts human transcribed transcripts of its standing committee meetings. 

## Installation

Please ensure StateLegiscraper is installed on your local drive.

## NVScrape

## NVProcess

## What Now?

You have data now – congratulations! This is where you, the user, have free reign to begin working with popular NLP Pythong packages, such as nltk and SpaCy. 

In [ ]:
# Word Frequency Example